In [ ]:
import os 
from glob import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.application import VGG16
from tensorflow.keras import layers
from tensorflow.keras.callbacks import Callbacks,Modelcheckpoint ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

In [ ]:
H, W = 256, 256
image_size = [256,256]
c=3
class_name = ["brain_glioma","brain_menin","brain_tumor"]
lr = 1e-4
model_path = ""

In [ ]:
path = "/kaggle/input/multi-cancer/Multi Cancer/Brain Cancer"

In [ ]:
def load_data(path,split=0.1):
    files = glob(os.path.join(path,"*","*"))
    split_rate = int(len(files) * split)
    
    train,valid = train_test_split(files,test_size=split_rate)
    train,test = train_test_split(train,test_size=split_rate)

    return train,valid,test

In [ ]:
files =load_data(path)

In [ ]:
def preprocess_data(image):
    img = cv2.imread(image,cv2.IMREAD_COLOR)
    img = cv2.resize(img,(H.W))
    img = img / 255.0
    img = img.astype(np.float32)

    lable = image.split("/")[-2]
    class_idx = classes_name.index[lable]

    return img,class_idx

In [ ]:
img,class_idx = preprocess_data(files[0])

In [ ]:
classes = np.array(class_idx,np.float32)

In [ ]:
def parse(path):
    images,labels = tf.numpy_function(preprocess_data,[path],[tf.float32,tf.int32])
    labels = tf.one_hot(labels,3)
    images.set_shape([256,256,3])
    labels.set_shape(3)

    return images,labels

In [ ]:
def tf_datasets(images, batch_size=8):
    dataset = tf.data.Dataset.from_tensor_slices((images))
    dataset = dataset.map(parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(8)
    return dataset

In [ ]:
train_ds = tf_datasets(train)
valid_ds = tf_datasets(valid)
test_ds = tf_datasets(test)

In [ ]:
image_ds = tf_datasets(files)

In [ ]:
for i,j in image_ds.take(1):
    print(i.numpy.shape())

In [ ]:
def PlotPipeImg(img_arr):
    fig,ax = plt.subplots(1,10,figsize=(10,10))
    axes = ax.flatten()
    for img, ax in zip(img_arr,axes):
        ax.imshow(img)
        ax.axis("off")

        plt.tight_layout()
        plt.show()

In [ ]:
img,idl = next(iter(image_ds))

In [ ]:
model = VGG16(input_shape=image_size+[c], weights='imagenet',include_top=False)

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
    layer.trainable = False

In [ ]:
x = layers.Flatten()(model.output)

In [ ]:
last_layer = layers.Dense(3,activation='softmax')(x)
model = Model(inputs=model.input,outputs=last_layer)

In [ ]:
callback=[
    Modelcheckpoint(model_path,verbose=1,save_best_only = True),
    ReduceLROnPlateau(monitor = "val_Loss",patience=5,min_lr = 1e-5,factor=0.1,verbose=1)
]

In [ ]:
model.compile(loss='',optimizer=Adam(lr),metrics=['accuracy'])

In [ ]:
model.fit(
    train_ds,
    valid_ds,
    epochs = 20,
    Callbacks,
)

In [ ]:
model.evaluate(test_ds)

In [ ]:
import itertools
def plt_confusion_matrix(cm, classes, normalize=False, title="Confusion Matrix", cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_mark = np.arange(len(classes))
    plt.xticks(tick_mark, classes, rotation=45)
    plt.yticks(tick_mark, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.axis]
        print("normalized confusion matrix")

    else:
        print("confusion matrix without normalization")

   thresh = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.xlabel("predicted label")
    plt.ylabel("True label")

In [ ]:
predictions = model.predict(test_ds)

In [ ]:
np.round(predictions)

In [ ]:
y_pred_classes = np.argmax(predictions, axis=1)

In [ ]:
def get_test_data_class(test_path):
    names = []
    for i in test_path:
        name = i.split("/")[-2]
        name_idx = class_name.index(name)
        names.append(name_idx)
    names = np.array(names, dtype=np.int32)
    return names

In [ ]:
classes = get_test_data_class(test)

In [ ]:
cm = confusion_matrix(y_true=classes, y_pred=y_pred_classes)

In [ ]:
plt_confusion_matrix(cm=cm, classes=class_name, title='cm')

In [ ]:
confusion matrix without normalization

In [ ]:
# Define a function to evaluate the model on a given dataset
def evaluate_model(model, dataset):
    y_true = []
    y_pred = []

    for images, labels in dataset:
        predictions = model.predict(images)
        predicted_labels = np.argmax(predictions, axis=1)
        true_labels = np.argmax(labels, axis=1)
        y_true.extend(true_labels)
        y_pred.extend(predicted_labels)

    accuracy = accuracy_score(y_true, y_pred)
    f1_scores = f1_score(y_true, y_pred, average=None)

    return accuracy, f1_scores

# Evaluate the model on the test dataset
test_accuracy, test_f1_scores = evaluate_model(model, test_ds)

# Evaluate the model on the train dataset (optional)
train_accuracy, train_f1_scores = evaluate_model(model, train_ds)

# Print the results
print("Test Accuracy:", test_accuracy)
print("F1-Score (Giloma):", test_f1_scores[0])
print("F1-Score (Menin):", test_f1_scores[1])
print("F1-Score (Tumor):", test_f1_scores[2])
